This notebook displays on the SPAM model of sulci the value of the parapeter for each sulcus. The entry point is a csv containing the value to plot for each sulcal region.

Requires anatomist from the BrainVISA software suite

# 1. Imports 

In [24]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [25]:
from soma import aims
import json
import os
import pandas as pd
import numpy as np

# 2. Paths and constants

In [26]:
# Path where the models and region configuration lie
path_to_deep_folding = "/neurospin/dico/data/deep_folding/current"

# JSON file with sulcal regions containing list of sulci
# json_regions = f"{path_to_deep_folding}/sulci_regions_gridsearch.json"
# json_regions = f"{path_to_deep_folding}/sulci_regions_champollion_V1.json"
json_regions = f"{os.getcwd()}"\
    "/../../view_gene_database/"\
    "region_to_sulci_with_smaller_region_size.json"

# Path to parameter file
path_summary = f"{path_to_deep_folding}/models/Champollion_V1_after_ablation/analysis/QTIM"
path_file = "IHI_QTIM_resid_sex_age.csv"
file_to_display = f"{path_summary}/{path_file}"

# Parameters and p value
param = "auc"
p_value = "p_value"
threshold = 0.05/56.
minVal = 0.5
maxVal = 1.0


In [27]:
# Gets SPAM models on which visualization is done
Rspam_model = aims.carto.Paths.findResourceFile(
    "models/models_2008/descriptive_models/"
    "segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg")
Lspam_model = aims.carto.Paths.findResourceFile(
    "models/models_2008/descriptive_models/segments/"
    "global_registered_spam_left/meshes/Lspam_model_meshes_1.arg")

# 2. Preprocessing

In [28]:
with open(json_regions) as f:
    regions = json.load(f)
print(len(regions))

50


In [29]:
# next(iter(regions[[brain]])) # Need the change to this if using the inital json file
next(iter(regions))

'F.C.L.p.-subsc.-F.C.L.a.-INSULA._left'

In [30]:
df = pd.read_csv(file_to_display)[["region", param, p_value]]
df.head()

,region,auc,p_value
0,F.Coll.-S.Rh._left,0.850495,0.000089
1,S.T.i.-S.O.T.lat._left,0.716641,0.000089
2,F.Coll.-S.Rh._right,0.696203,0.000089
3,S.T.i.-S.O.T.lat._right,0.692656,0.000089
4,F.P.O.-S.Cu.-Sc.Cal._left,0.688620,0.000089


In [31]:
res = df.set_index("region")
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [32]:
def get_sulci(region):
    if region in regions:
        return regions[f"{region}"]
    else:
        return []

In [33]:
get_sulci("S.C.-sylv._left")

['S.C._left', 'S.C.sylvian._left']

In [34]:
len(regions.keys())

50

In [35]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,auc,p_value,side,sulcus
0,F.Coll.-S.Rh._left,0.850495,0.000089,left,"[F.Coll._left, S.Rh._left]"
1,S.T.i.-S.O.T.lat._left,0.716641,0.000089,left,"[S.O.T.lat.ant._left, S.O.T.lat.int._left, S.O..."
2,F.Coll.-S.Rh._right,0.696203,0.000089,right,"[F.Coll._right, S.Rh._right]"
3,S.T.i.-S.O.T.lat._right,0.692656,0.000089,right,"[S.O.T.lat.ant._right, S.O.T.lat.int._right, S..."
4,F.P.O.-S.Cu.-Sc.Cal._left,0.688620,0.000089,left,"[F.P.O._left, S.Cu._left, S.Pa.t._left]"


In [36]:
res[res["sulcus"].str.len() == 0]

,region,auc,p_value,side,sulcus
14,S.C.-S.Pe.C._right,0.593671,0.002053,right,[]
24,S.C.-S.Po.C._right,0.562229,0.038300,right,[]
43,S.C.-S.Pe.C._left,0.505049,0.430587,left,[]
44,F.I.P.-F.I.P.Po.C.inf._left,0.503647,0.461298,left,[]
45,F.I.P.-F.I.P.Po.C.inf._right,0.502115,0.476297,right,[]
51,S.C.-S.Po.C._left,0.492078,0.583876,left,[]


In [37]:
res = res.sort_values(by=param, ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,auc,p_value,side,sulcus
6,S.T.i.-S.T.s.-S.T.pol._left,0.652506,0.000179,left,S.T.pol._left
10,S.T.s._left,0.629113,0.000268,left,S.T.s._left
26,S.T.s._right,0.558147,0.072136,right,S.T.s._right
34,S.T.i.-S.T.s.-S.T.pol._right,0.536855,0.172395,right,S.T.pol._right
36,S.T.s.br._left,0.529865,0.186680,left,F.I.P.r.int.1_left
36,S.T.s.br._left,0.529865,0.186680,left,F.I.P.r.int.2_left
36,S.T.s.br._left,0.529865,0.186680,left,S.GSM._left
36,S.T.s.br._left,0.529865,0.186680,left,S.T.s.ter.asc.ant._left
36,S.T.s.br._left,0.529865,0.186680,left,S.T.s.ter.asc.post._left
50,S.T.s.br._right,0.493057,0.576288,right,F.I.P.r.int.1_right


In [38]:
res[res["sulcus"].str.len() == 0]

,region,auc,p_value,side,sulcus


In [50]:
res[param].tolist()

[0.8504950079808514,
 0.8504950079808514,
 0.7166407063007402,
 0.7166407063007402,
 0.7166407063007402,
 0.7166407063007402,
 0.7166407063007402,
 0.7166407063007402,
 0.696202521807576,
 0.696202521807576,
 0.6926556081217279,
 0.6926556081217279,
 0.6926556081217279,
 0.6926556081217279,
 0.6926556081217279,
 0.6926556081217279,
 0.6886202435954812,
 0.6886202435954812,
 0.6886202435954812,
 0.6862223757514773,
 0.6862223757514773,
 0.6862223757514773,
 0.6862223757514773,
 0.6525062170959306,
 0.6496946987130938,
 0.6496946987130938,
 0.6496946987130938,
 0.6339499807634642,
 0.6339499807634642,
 0.631145116758524,
 0.631145116758524,
 0.631145116758524,
 0.631145116758524,
 0.631145116758524,
 0.6291134641099797,
 0.6225767138687908,
 0.6053137837751111,
 0.6001607849836882,
 0.5936711028152285,
 0.5900586710755834,
 0.5900586710755834,
 0.5861538168978517,
 0.5861538168978517,
 0.5843233368318901,
 0.5843233368318901,
 0.5843233368318901,
 0.580747960515266,
 0.580747960515266,
 

no position could be read at 2, 148
Position : 65.6347, 39.3269, -49.3053, 0
Position : 33.352, 54.9687, -71.6552, 0


In [39]:
# for _, row in res.iterrows():
#     print(row.sulcus)

In [40]:
# res[res.sulcus=="S.F.orbitaire._right"]

In [41]:
# res

# 3. Anatomist functions

In [51]:
def set_color_property(res, side):
    global dic
    global param, p_value, threshold

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex[param] = 0.
    print(f"param = {param}")
        
    unknown_vertices = []
    for vertex in dic[f"aims{side}"].vertices():
        vname = vertex.get('name')
        if vname == 'unknown':
            #print(f"Removing vertex with name: {name}")
            unknown_vertices.append(vertex)
    for vertex in unknown_vertices:
        dic[f"aims{side}"].removeVertex(vertex)

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                if row[p_value] < threshold:
                    vertex[param] = row[param]
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty(param)
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=minVal, maxVal=maxVal,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


# Main function

In [52]:
a = anatomist.Anatomist()

In [53]:
middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]

In [54]:
%matplotlib qt5

In [55]:
dic = {} # Global dictionary of windows

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
param = auc
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
param = auc
bounding box found : -90, -80, -90
                     90, 120, 60
